In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, RadioButtons, Button, CheckButtons
from matplotlib.patches import Rectangle, Circle, Arrow, Polygon
import matplotlib.gridspec as gridspec

# Initial parameters
wheelbase = 2.5  # L (m)
track_width = 1.5  # B (m)
turning_radius = 5.0  # R (m) to outside wheel
max_steering_angle = 40.0  # Maximum steering angle in degrees
steering_ratio = 15.0  # Steering wheel to road wheel ratio
kingpin_inclination = 10.0  # Degrees
caster_angle = 5.0  # Degrees
scrub_radius = 0.05  # meters

# Function to calculate the corresponding angle using the Ackermann relationship
def calculate_inner_outer_angles(turning_radius, wheelbase, track_width):
    """Calculate both inner and outer angles based on turning radius"""
    # For a given turning radius to the center of the rear axle
    # Geometry: turning radius is measured to the center of the rear axle
    R = turning_radius
    L = wheelbase
    B = track_width
    
    # Calculate angles based on the turning geometry
    # Inner angle is larger than outer angle
    # Inner wheel angle (δi)
    inner_angle_rad = np.arctan(L / (R - B/2))
    # Outer wheel angle (δo)
    outer_angle_rad = np.arctan(L / (R + B/2))
    
    return np.rad2deg(inner_angle_rad), np.rad2deg(outer_angle_rad)

def calculate_outer_angle(inner_angle, B, L):
    """Calculate outer angle from inner angle using Ackermann relationship"""
    # Convert degrees to radians
    inner_rad = np.deg2rad(inner_angle)
    # Apply the Ackermann relationship: cot(δo) - cot(δi) = B/L
    cot_inner = 1 / np.tan(inner_rad)
    cot_outer = cot_inner + B/L
    # Handle potential numerical issues
    if np.abs(cot_outer) < 1e-10:
        outer_rad = np.pi/2 if cot_outer >= 0 else -np.pi/2
    else:
        outer_rad = np.arctan(1 / cot_outer)
    return np.rad2deg(outer_rad)

def calculate_turning_radius(inner_angle, B, L):
    """Calculate turning radius to center of rear axle from inner angle"""
    inner_rad = np.deg2rad(inner_angle)
    # Inner wheel radius: Ri = L/tan(δi)
    Ri = L / np.tan(inner_rad)
    # Distance from center of turn to centerline of vehicle
    R = Ri + B/2
    return R

def calculate_ideal_ackermann_angle(steering_angle, B, L, side='inner'):
    """Calculate ideal Ackermann angle for a given steering input"""
    steering_rad = np.deg2rad(steering_angle)
    R = L / np.tan(steering_rad)  # Radius to center of turn
    
    if side == 'inner':
        return np.rad2deg(np.arctan(L / (R - B/2)))
    else:  # outer
        return np.rad2deg(np.arctan(L / (R + B/2)))

def calculate_ackermann_percentage(inner_angle, outer_angle, B, L):
    """Calculate how close the steering is to perfect Ackermann geometry"""
    # First calculate what the ideal outer angle should be for this inner angle
    ideal_outer = calculate_outer_angle(inner_angle, B, L)
    
    # Calculate the deviation from ideal
    deviation = np.abs(ideal_outer - outer_angle)
    
    # Calculate the maximum possible deviation (parallel steering)
    max_deviation = np.abs(ideal_outer - inner_angle)
    
    if max_deviation == 0:  # Avoid division by zero
        return 100.0
    
    # Return the percentage of Ackermann (100% = perfect Ackermann, 0% = parallel steering)
    return (1 - deviation / max_deviation) * 100.0

def plot_vehicle(ax, wheelbase, track_width, inner_angle, outer_angle, turning_radius, 
                 show_turning_circle=True, show_ackermann_arms=False,
                 kingpin_inclination=10.0, caster_angle=5.0, scrub_radius=0.05):
    """Draw the vehicle with wheels at the correct angles"""
    ax.clear()
    # Plot parameters
    car_length = wheelbase * 1.3  # Car extends beyond wheelbase
    car_width = track_width * 1.1
    wheel_width = track_width * 0.1
    wheel_length = wheelbase * 0.1
    
    # Draw car body
    car_body = Rectangle((-car_length * 0.3, -car_width/2), car_length, car_width, 
                        fill=False, edgecolor='blue', linewidth=2)
    ax.add_patch(car_body)
    
    # Draw wheels
    # Front left wheel (inner wheel during right turn)
    fl_wheel = Rectangle((wheelbase * 0.7 - wheel_length/2, track_width/2 - wheel_width/2), 
                        wheel_length, wheel_width, 
                        angle=inner_angle, fill=True, color='black')
    # Front right wheel (outer wheel during right turn)
    fr_wheel = Rectangle((wheelbase * 0.7 - wheel_length/2, -track_width/2 - wheel_width/2), 
                        wheel_length, wheel_width, 
                        angle=outer_angle, fill=True, color='black')
    # Rear wheels (straight)
    rl_wheel = Rectangle((-wheelbase * 0.3 - wheel_length/2, track_width/2 - wheel_width/2), 
                        wheel_length, wheel_width, fill=True, color='black')
    rr_wheel = Rectangle((-wheelbase * 0.3 - wheel_length/2, -track_width/2 - wheel_width/2), 
                        wheel_length, wheel_width, fill=True, color='black')
    
    ax.add_patch(fl_wheel)
    ax.add_patch(fr_wheel)
    ax.add_patch(rl_wheel)
    ax.add_patch(rr_wheel)
    
    # Draw kingpin axis indicators if requested 
    if show_ackermann_arms:
        # Length of the steering arm visualization
        arm_length = track_width * 0.4
        
        # Calculate steering arm endpoints for left wheel
        left_start = (wheelbase * 0.7, track_width/2)
        left_end = (wheelbase * 0.7 - arm_length * np.sin(np.deg2rad(inner_angle)), 
                   track_width/2 - arm_length * np.cos(np.deg2rad(inner_angle)))
        
        # Calculate steering arm endpoints for right wheel
        right_start = (wheelbase * 0.7, -track_width/2)
        right_end = (wheelbase * 0.7 - arm_length * np.sin(np.deg2rad(outer_angle)), 
                    -track_width/2 - arm_length * np.cos(np.deg2rad(outer_angle)))
        
        # Draw steering arms
        ax.plot([left_start[0], left_end[0]], [left_start[1], left_end[1]], 'r-', linewidth=2)
        ax.plot([right_start[0], right_end[0]], [right_start[1], right_end[1]], 'r-', linewidth=2)
        
        # Calculate and draw tie rod (simplified)
        ax.plot([left_end[0], right_end[0]], [left_end[1], right_end[1]], 'g-', linewidth=2)
    
    # Draw turning center and turning radius
    if show_turning_circle:
        # Calculate coordinates of turning center
        x_tc = -turning_radius
        y_tc = 0
        # Draw turning center point
        ax.add_patch(Circle((x_tc, y_tc), 0.1, color='red'))
        
        # Draw turning radii lines
        # To center of rear axle
        ax.plot([x_tc, -wheelbase * 0.3], [y_tc, 0], 'r--', alpha=0.7)
        # To inner wheel
        ax.plot([x_tc, wheelbase * 0.7], [y_tc, track_width/2], 'r--', alpha=0.7)
        # To outer wheel
        ax.plot([x_tc, wheelbase * 0.7], [y_tc, -track_width/2], 'r--', alpha=0.7)
        
        # Draw turning circle
        turning_circle = Circle((x_tc, y_tc), turning_radius, fill=False, color='red', alpha=0.3)
        ax.add_patch(turning_circle)
    
    # Set axis properties
    max_dim = max(car_length, car_width) + turning_radius + 2
    ax.set_xlim(-turning_radius - 1, car_length + 1)
    ax.set_ylim(-max_dim/2, max_dim/2)
    ax.set_aspect('equal')
    ax.grid(True)
    ax.set_title('Vehicle Top-Down View')
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    
    # Calculate Ackermann percentage
    ackermann_pct = calculate_ackermann_percentage(inner_angle, outer_angle, track_width, wheelbase)
    
    # Add text with angles and turning radius
    text_info = f"Inner angle: {inner_angle:.1f}°\nOuter angle: {outer_angle:.1f}°\n"
    text_info += f"Turning radius: {turning_radius:.1f}m\nTrack width: {track_width:.1f}m\nWheelbase: {wheelbase:.1f}m\n"
    text_info += f"Ackermann %: {ackermann_pct:.1f}%"
    
    # Add 3D parameters if requested
    if show_ackermann_arms:
        text_info += f"\nKingpin inclination: {kingpin_inclination:.1f}°\nCaster angle: {caster_angle:.1f}°\n"
        text_info += f"Scrub radius: {scrub_radius*1000:.1f}mm"
    
    ax.text(0.02, 0.98, text_info, transform=ax.transAxes,
            verticalalignment='top', horizontalalignment='left',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

def plot_3d_wheel(ax, kingpin_inclination, caster_angle, scrub_radius):
    """Draw a simple 3D representation of the wheel showing steering geometry"""
    ax.clear()
    
    # Set up 3D plot with proper scaling
    wheel_radius = 0.3  # Wheel radius in meters 
    wheel_width = 0.2   # Wheel width in meters
    
    # Draw wheel as a simple cylinder (simplified)
    u = np.linspace(0, 2 * np.pi, 20)
    v = np.linspace(0, np.pi, 10)
    x = wheel_radius * np.outer(np.cos(u), np.sin(v))
    y = wheel_width/2 * np.outer(np.ones(np.size(u)), np.ones(np.size(v)))
    z = wheel_radius * np.outer(np.sin(u), np.sin(v))
    
    # Adjust for orientation
    ax.plot_surface(z, x, y, color='gray', alpha=0.5)
    
    # Calculate kingpin axis endpoints
    kingpin_length = wheel_radius * 2.5
    kingpin_rad = np.deg2rad(kingpin_inclination)
    caster_rad = np.deg2rad(caster_angle)
    
    # Start point (upper) of kingpin axis
    kp_start_x = -kingpin_length/2 * np.sin(kingpin_rad)
    kp_start_y = 0
    kp_start_z = kingpin_length/2 * np.cos(kingpin_rad)
    
    # End point (lower) of kingpin axis
    kp_end_x = kingpin_length/2 * np.sin(kingpin_rad)
    kp_end_y = kingpin_length/2 * np.sin(caster_rad)
    kp_end_z = -kingpin_length/2 * np.cos(kingpin_rad)
    
    # Draw kingpin axis
    ax.plot([kp_start_z, kp_end_z], [kp_start_x, kp_end_x], [kp_start_y, kp_end_y], 'r-', linewidth=3)
    
    # Draw scrub radius
    contact_point_x = 0
    contact_point_y = 0
    contact_point_z = -wheel_radius
    
    # Point where kingpin axis intersects ground
    ground_intersection_x = scrub_radius
    ground_intersection_y = 0  
    ground_intersection_z = -wheel_radius
    
    # Draw ground plane (square)
    ground_size = wheel_radius * 3
    xx, yy = np.meshgrid(np.linspace(-ground_size, ground_size, 10), 
                         np.linspace(-ground_size, ground_size, 10))
    zz = np.ones_like(xx) * (-wheel_radius)
    ax.plot_surface(zz, xx, yy, color='green', alpha=0.2)
    
    # Draw scrub radius line
    ax.plot([contact_point_z, ground_intersection_z], 
            [contact_point_x, ground_intersection_x], 
            [contact_point_y, ground_intersection_y], 'b-', linewidth=2)
    
    # Annotate the plot
    ax.text(contact_point_z, contact_point_x, contact_point_y, "Contact Point", color='black')
    ax.text(ground_intersection_z, ground_intersection_x, ground_intersection_y, 
            f"Scrub Radius: {scrub_radius*1000:.1f}mm", color='blue')
    ax.text(kp_start_z, kp_start_x, kp_start_y, f"Kingpin\nInclination: {kingpin_inclination:.1f}°", color='red')
    ax.text(kp_end_z, kp_end_x, kp_end_y, f"Caster Angle: {caster_angle:.1f}°", color='red')
    
    # Set axis properties
    ax.set_xlabel('Z')
    ax.set_ylabel('X')
    ax.set_zlabel('Y')
    ax.set_title('Wheel Steering Geometry (Front View)')
    ax.set_box_aspect([1, 1, 1])  # Equal aspect ratio
    
    # Set limits for better visualization
    ax.set_xlim(-wheel_radius * 2, wheel_radius * 2)
    ax.set_ylim(-wheel_radius * 2, wheel_radius * 2)
    ax.set_zlim(-wheel_radius * 2, wheel_radius * 2)

# Create the figure and subplots
fig = plt.figure(figsize=(18, 12))
gs = gridspec.GridSpec(3, 3, height_ratios=[1, 1, 2])

ax_graph = fig.add_subplot(gs[0, 0])        # Top left - angle relationship graph
ax_turning = fig.add_subplot(gs[0, 1])      # Top middle - turning radius vs inner angle
ax_3d = fig.add_subplot(gs[0, 2], projection='3d')  # Top right - 3D wheel geometry
ax_ackermann = fig.add_subplot(gs[1, 0])    # Middle left - Ackermann percentage
ax_steer_ratio = fig.add_subplot(gs[1, 1])  # Middle middle - Steering ratio effects
ax_controls = fig.add_subplot(gs[1, 2])     # Middle right - Instructions/controls
ax_vehicle = fig.add_subplot(gs[2, :])      # Bottom full width - vehicle visualization

plt.subplots_adjust(bottom=0.25, wspace=0.3, hspace=0.3)

# Calculate initial angles based on turning radius
inner_angle_init, outer_angle_init = calculate_inner_outer_angles(turning_radius, wheelbase, track_width)

# Set up angle relationship graph (top left)
inner_angles = np.linspace(1, 45, 100)  # degrees
outer_angles = [calculate_outer_angle(inner, track_width, wheelbase) for inner in inner_angles]
parallel_steering = inner_angles  # For comparison (parallel steering has equal angles)
angle_plot, = ax_graph.plot(inner_angles, outer_angles, 'b-', linewidth=2, label='Ackermann')
parallel_plot, = ax_graph.plot(inner_angles, parallel_steering, 'k--', alpha=0.5, label='Parallel')
angle_marker, = ax_graph.plot([inner_angle_init], [outer_angle_init], 'ro', markersize=8)

ax_graph.set_xlabel('Inner Steering Angle (degrees)')
ax_graph.set_ylabel('Outer Steering Angle (degrees)')
ax_graph.set_title('Ackermann Steering Angle Relationship')
ax_graph.grid(True)
ax_graph.legend()
ax_graph.set_xlim(0, 45)
ax_graph.set_ylim(0, 45)

# Set up turning radius graph (top right)
turning_radii = [calculate_turning_radius(inner, track_width, wheelbase) for inner in inner_angles]
radius_plot, = ax_turning.plot(inner_angles, turning_radii, 'g-', linewidth=2)
radius_marker, = ax_turning.plot([inner_angle_init], [turning_radius], 'ro', markersize=8)

ax_turning.set_xlabel('Inner Steering Angle (degrees)')
ax_turning.set_ylabel('Turning Radius (m)')
ax_turning.set_title('Turning Radius vs Inner Angle')
ax_turning.grid(True)
ax_turning.set_xlim(0, 45)
ax_turning.set_ylim(0, 20)

# Set up Ackermann percentage graph (middle left)
ackermann_pcts = [calculate_ackermann_percentage(inner, calculate_outer_angle(inner, track_width, wheelbase), 
                                             track_width, wheelbase) for inner in inner_angles]
ackermann_plot, = ax_ackermann.plot(inner_angles, ackermann_pcts, 'b-', linewidth=2)
ackermann_marker, = ax_ackermann.plot([inner_angle_init], 
                                     [calculate_ackermann_percentage(inner_angle_init, outer_angle_init, 
                                                                   track_width, wheelbase)], 'ro', markersize=8)

ax_ackermann.set_xlabel('Inner Steering Angle (degrees)')
ax_ackermann.set_ylabel('Ackermann Percentage (%)')
ax_ackermann.set_title('Ackermann Compliance')
ax_ackermann.grid(True)
ax_ackermann.set_xlim(0, 45)
ax_ackermann.set_ylim(0, 110)

# Setup steering ratio plot (middle middle)
steering_wheel_angles = np.linspace(0, 720, 100)  # Steering wheel angle (degrees)
road_wheel_angles = [min(angle/steering_ratio, max_steering_angle) for angle in steering_wheel_angles]
steer_ratio_plot, = ax_steer_ratio.plot(steering_wheel_angles, road_wheel_angles, 'm-', linewidth=2)
steer_ratio_marker, = ax_steer_ratio.plot([inner_angle_init*steering_ratio], [inner_angle_init], 'ro', markersize=8)

ax_steer_ratio.set_xlabel('Steering Wheel Angle (degrees)')
ax_steer_ratio.set_ylabel('Road Wheel Angle (degrees)')
ax_steer_ratio.set_title(f'Steering Ratio: {steering_ratio:.1f}:1')
ax_steer_ratio.grid(True)
ax_steer_ratio.set_xlim(0, 720)
ax_steer_ratio.set_ylim(0, max_steering_angle*1.1)

# Controls panel with instructions
ax_controls.axis('off')
instructions = "Ackermann Steering Simulator\n\n"
instructions += "• Use 'Control Mode' to select what parameter to adjust\n"
instructions += "• Adjust sliders to change vehicle parameters\n"
instructions += "• Toggle visualization options with checkboxes\n"
instructions += "• Reset button returns to default values\n"
instructions += "• Save button saves the current figure\n\n"
instructions += "Ackermann steering is a geometric arrangement of linkages\n"
instructions += "that ensures the inner wheel turns at a sharper angle\n"
instructions += "than the outer wheel during a turn, reducing tire scrub."
ax_controls.text(0.5, 0.5, instructions, 
                ha='center', va='center', 
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))

# Plot the 3D wheel geometry
plot_3d_wheel(ax_3d, kingpin_inclination, caster_angle, scrub_radius)

# Initial vehicle plot
plot_vehicle(ax_vehicle, wheelbase, track_width, inner_angle_init, outer_angle_init, turning_radius)

# Add sliders for parameters
slider_color = 'lightblue'
slider_width = 0.65
slider_height = 0.025
slider_left = 0.20
slider_spacing = 0.035

# Basic vehicle parameters
y_pos = 0.20
ax_turning_radius = plt.axes([slider_left, y_pos, slider_width, slider_height], facecolor=slider_color)
turning_radius_slider = Slider(ax_turning_radius, 'Turning Radius (m)', 2.0, 20.0, valinit=turning_radius)

y_pos -= slider_spacing
ax_track_width = plt.axes([slider_left, y_pos, slider_width, slider_height], facecolor=slider_color)
track_width_slider = Slider(ax_track_width, 'Track Width (m)', 0.5, 3.0, valinit=track_width)

y_pos -= slider_spacing
ax_wheelbase = plt.axes([slider_left, y_pos, slider_width, slider_height], facecolor=slider_color)
wheelbase_slider = Slider(ax_wheelbase, 'Wheelbase (m)', 1.0, 5.0, valinit=wheelbase)

y_pos -= slider_spacing
ax_inner_angle = plt.axes([slider_left, y_pos, slider_width, slider_height], facecolor=slider_color)
inner_angle_slider = Slider(ax_inner_angle, 'Inner Angle (°)', 1.0, 45.0, valinit=inner_angle_init)

# Advanced parameters
y_pos -= slider_spacing * 1.2  # Extra space before advanced parameters
ax_steering_ratio = plt.axes([slider_left, y_pos, slider_width, slider_height], facecolor=slider_color)
steering_ratio_slider = Slider(ax_steering_ratio, 'Steering Ratio', 5.0, 25.0, valinit=steering_ratio)

y_pos -= slider_spacing
ax_kingpin = plt.axes([slider_left, y_pos, slider_width, slider_height], facecolor=slider_color)
kingpin_slider = Slider(ax_kingpin, 'Kingpin Inclination (°)', 0.0, 20.0, valinit=kingpin_inclination)

y_pos -= slider_spacing
ax_caster = plt.axes([slider_left, y_pos, slider_width, slider_height], facecolor=slider_color)
caster_slider = Slider(ax_caster, 'Caster Angle (°)', 0.0, 15.0, valinit=caster_angle)

y_pos -= slider_spacing
ax_scrub = plt.axes([slider_left, y_pos, slider_width, slider_height], facecolor=slider_color)
scrub_slider = Slider(ax_scrub, 'Scrub Radius (mm)', -50.0, 100.0, valinit=scrub_radius*1000)

# Radio buttons for control mode (what parameter to focus on changing)
ax_radio = plt.axes([0.05, 0.12, 0.1, 0.08])
radio = RadioButtons(ax_radio, ('Radius', 'Inner Angle'), active=0)

# Checkboxes for visualization options
ax_check = plt.axes([0.05, 0.03, 0.1, 0.06])
check = CheckButtons(ax_check, ['Turning Circle', 'Steering Arms'], [True, False])

# Add Reset and Save buttons
ax_reset = plt.axes([0.05, 0.20, 0.1, 0.04])
reset_button = Button(ax_reset, 'Reset', color=slider_color, hovercolor='0.9')

ax_save = plt.axes([0.05, 0.25, 0.1, 0.04])
save_button = Button(ax_save, 'Save Figure', color=slider_color, hovercolor='0.9')

# Global variables to prevent recursive callbacks
updating = False
mode = 'Radius'  # Initial mode
show_turning_circle = True
show_steering_arms = False

def update_plots():
    """Update all plots based on current parameter values"""
    global updating, wheelbase, track_width, turning_radius, kingpin_inclination, caster_angle, scrub_radius, steering_ratio
    
    if updating:
        return  # Avoid recursion
    
    updating = True  # Set lock
    
    # Get current values
    track_width = track_width_slider.val
    wheelbase = wheelbase_slider.val
    kingpin_inclination = kingpin_slider.val
    caster_angle = caster_slider.val
    scrub_radius = scrub_slider.val / 1000  # Convert from mm to m
    steering_ratio = steering_ratio_slider.val
    
    if mode == 'Radius':
        # In radius mode, turning radius controls everything
        turning_radius = turning_radius_slider.val
        inner_angle, outer_angle = calculate_inner_outer_angles(turning_radius, wheelbase, track_width)
        # Update inner angle slider without triggering callback
        inner_angle_slider.set_val(inner_angle)
    else:  # 'Inner Angle'
        # In inner angle mode, inner angle controls everything
        inner_angle = inner_angle_slider.val
        turning_radius = calculate_turning_radius(inner_angle, track_width, wheelbase)
        outer_angle = calculate_outer_angle(inner_angle, track_width, wheelbase)
        # Update turning radius slider without triggering callback
        turning_radius_slider.set_val(turning_radius)
    
    # Update angle graph
    outer_angles = [calculate_outer_angle(inner, track_width, wheelbase) for inner in inner_angles]
    angle_plot.set_ydata(outer_angles)
    angle_marker.set_data([inner_angle], [outer_angle])
    
    # Update turning radius graph
    turning_radii = [calculate_turning_radius(inner, track_width, wheelbase) for inner in inner_angles]
    radius_plot.set_ydata(turning_radii)
    radius_marker.set_data([inner_angle], [turning_radius])
    
    # Update Ackermann percentage graph
    ackermann_pcts = [calculate_ackermann_percentage(inner, calculate_outer_angle(inner, track_width, wheelbase), 
                                                 track_width, wheelbase) for inner in inner_angles]
    ackermann_plot.set_ydata(ackermann_pcts)
    current_ackermann_pct = calculate_ackermann_percentage(inner_angle, outer_angle, track_width, wheelbase)
    ackermann_marker.set_data([inner_angle], [current_ackermann_pct])
    
    # Update steering ratio plot
    road_wheel_angles = [min(angle/steering_ratio, max_steering_angle) for angle in steering_wheel_angles]
    steer_ratio_plot.set_ydata(road_wheel_angles)
    ax_steer_ratio.set_title(f'Steering Ratio: {steering_ratio:.1f}:1')
    steer_ratio_marker.set_data([inner_angle*steering_ratio], [inner_angle])
    
    # Update 3D wheel geometry
    plot_3d_wheel(ax_3d, kingpin_inclination, caster_angle, scrub_radius)
    
    # Update vehicle visualization
    plot_vehicle(ax_vehicle, wheelbase, track_width, inner_angle, outer_angle, turning_radius,
                show_turning_circle, show_steering_arms,
                kingpin_inclination, caster_angle, scrub_radius)
    
    fig.canvas.draw_idle()
    updating = False  # Release lock

def on_radius_change(val):
    """Callback for turning radius slider"""
    if mode == 'Radius':  # Only respond when in Radius mode
        update_plots()

def on_inner_angle_change(val):
    """Callback for inner angle slider"""
    if mode == 'Inner Angle':  # Only respond when in Inner Angle mode
        update_plots()

def on_param_change(val):
    """Callback for track width and wheelbase sliders"""
    update_plots()

def on_mode_change(label):
    """Callback for radio button mode selection"""
    global mode
    mode = label
    update_plots()

def on_visual_change(label):
    """Callback for visualization checkboxes"""
    global show_turning_circle, show_steering_arms
    if label == 'Turning Circle':
        show_turning_circle = not show_turning_circle
    elif label == 'Steering Arms':
        show_steering_arms = not show_steering_arms
    update_plots()

def on_reset(event):
    """Reset all parameters to initial values"""
    turning_radius_slider.reset()
    track_width_slider.reset()
    wheelbase_slider.reset()
    inner_angle_slider.reset()
    steering_ratio_slider.reset()
    kingpin_slider.reset()
    caster_slider.reset()
    scrub_slider.reset()
    update_plots()

def on_save(event):
    """Save the current figure"""
    plt.savefig('ackermann_steering_simulation.png', dpi=300, bbox_inches='tight')
    plt.savefig('ackermann_steering_simulation.pdf', bbox_inches='tight')
    print("Figure saved as 'ackermann_steering_simulation.png' and 'ackermann_steering_simulation.pdf'")

# Connect sliders to their respective callback functions
turning_radius_slider.on_changed(on_radius_change)
inner_angle_slider.on_changed(on_inner_angle_change)
track_width_slider.on_changed(on_param_change